<a target="_blank" href="https://colab.research.google.com/github/mtan-unibie/GIGASci">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Install packages

In [ ]:
!pip install torch
!pip install torchvision
!pip install tensorboard
!pip install pillow
!pip install timm==0.3.2
!pip install numpy
!pip install seaborn
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip install requests
!pip install wget
!pip install torchmetrics

import requests
import wget
import subprocess
import os

#### a fix to work with PyTorch 1.8.1+

In [ ]:
result = subprocess.run(['pip', 'show', 'timm'], capture_output=True, text=True)
output_lines = result.stdout.split('\n')
location = None
for line in output_lines:
    if line.startswith('Location:'):
        location = line.split(': ')[1]

In [ ]:
%%writefile {os.path.join(location, 'timm/models/layers/helpers.py')}
""" Layer/Module Helpers

Hacked together by / Copyright 2020 Ross Wightman
"""
from itertools import repeat
#from torch._six import container_abcs
import collections.abc as container_abcs


# From PyTorch internals
def _ntuple(n):
    def parse(x):
        if isinstance(x, container_abcs.Iterable):
            return x
        return tuple(repeat(x, n))
    return parse


to_1tuple = _ntuple(1)
to_2tuple = _ntuple(2)
to_3tuple = _ntuple(3)
to_4tuple = _ntuple(4)
to_ntuple = _ntuple

### download python files

In [ ]:
!git clone 'https://github.com/facebookresearch/mae.git'
%cd mae

# URL of the .patch file
url = "https://anonymous.4open.science/api/repo/MAE-ViT-on-diatom-classification-5CB1/file/modifications.patch?v=1cd77079b9ff69c1f00e181ec0a161e215eccffe&download=true"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Save the content of the response to a file
    with open('modifications.patch', 'wb') as f:
        f.write(response.content)
    print("Patch file downloaded successfully.")
else:
    print("Failed to download patch file.")
    
!git apply modifications.patch

### download data & build testset

In [ ]:
!curl -u "TBLSXLnL4f8r6iJ:DiatomsRock" -H 'X-Requested-With: XMLHttpRequest' https://nxcl.biologie.uni-due.de/public.php/webdav --output UDE_Diatoms_in_the_Wild_2024.zip
!unzip 'UDE_Diatoms_in_the_Wild_2024.zip'  
!python testset.py   

### download pre-trained checkpoints

In [ ]:
%mkdir 'checkpoints'
%cd 'checkpoints'
wget.download('https://docs.cebitec.uni-bielefeld.de/s/J466HPSoPzXRQ6D/download?path=%2F&files=mae_cls144_0.1_ep100_checkpoint.pth')
wget.download('https://docs.cebitec.uni-bielefeld.de/s/J466HPSoPzXRQ6D/download?path=%2F&files=mae_cls144_ep100_checkpoint.pth')
wget.download('https://docs.cebitec.uni-bielefeld.de/s/J466HPSoPzXRQ6D/download?path=%2F&files=vit_cls144_0.1_ep100_checkpoint.pth')
wget.download('https://docs.cebitec.uni-bielefeld.de/s/J466HPSoPzXRQ6D/download?path=%2F&files=vit_cls144_ep100_checkpoint.pth')
%cd ..

### evaluation

#### ViT

In [ ]:
!python ViT-classification.py --eval --device cuda --batch_size 16 --nb_classes 144 --resume checkpoints/vit_cls144_ep100_checkpoint.pth --data_path Testset

In [ ]:
!python ViT-classification.py --eval --device cuda --batch_size 16 --nb_classes 144 --resume checkpoints/vit_cls144_0.1_ep100_checkpoint.pth --data_path Testset

#### MAE

In [ ]:
!python main_finetune.py  --eval --device cuda --batch_size 16 --nb_classes 144 --resume checkpoints/mae_cls144_ep100_checkpoint.pth --data_path Testset

In [ ]:
!python main_finetune.py  --eval --device cuda --batch_size 16 --nb_classes 144 --resume checkpoints/mae_cls144_0.1_ep100_checkpoint.pth --data_path Testset